### Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

### In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

#### The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

In [8]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

soup = BeautifulSoup(raw_wikipedia_page,'lxml')
#print(soup.prettify())


In [9]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)
    

In [10]:
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)


In [6]:
!pip install geocoder
print('geocoder has been installed before.')
import geocoder
print('geocoder has been successfully imported.')

geocoder has been installed before.
geocoder has been successfully imported.


In [31]:
latitude = []
longitude = []
for elem in Postcode_u:
# initialize your variable to None
    lat_lng_coords = None

# loop until you get the coordinates
    while (lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(elem))
        lat_lng_coords = g.latlng
        # print(lat_lng_coords)

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    print(elem, 'is RECEIVED.')
    # print(lat_lng_coords[0])
    # print(lat_lng_coords[1])
    

M4X is RECEIVED.
M4G is RECEIVED.
M9M is RECEIVED.
M1W is RECEIVED.
M1J is RECEIVED.
M9P is RECEIVED.
M1G is RECEIVED.
M4J is RECEIVED.
M8Y is RECEIVED.
M5A is RECEIVED.
M1C is RECEIVED.
M5L is RECEIVED.
M8V is RECEIVED.
M6R is RECEIVED.
M8Z is RECEIVED.
M5M is RECEIVED.
M1N is RECEIVED.
M2H is RECEIVED.
M4C is RECEIVED.
M3K is RECEIVED.
M6P is RECEIVED.
M9C is RECEIVED.
M4B is RECEIVED.
M4E is RECEIVED.
M4K is RECEIVED.
M9R is RECEIVED.
M6L is RECEIVED.
M4V is RECEIVED.
M5X is RECEIVED.
M4L is RECEIVED.
M9L is RECEIVED.
M1R is RECEIVED.
M6A is RECEIVED.
M3C is RECEIVED.
M1V is RECEIVED.
M5K is RECEIVED.
M1H is RECEIVED.
M3H is RECEIVED.
M9N is RECEIVED.
M6B is RECEIVED.
M6M is RECEIVED.
M1T is RECEIVED.
M5B is RECEIVED.
M8W is RECEIVED.
M5P is RECEIVED.
M5T is RECEIVED.
M3L is RECEIVED.
M3A is RECEIVED.
M9A is RECEIVED.
M1M is RECEIVED.
M4H is RECEIVED.
M5C is RECEIVED.
M6J is RECEIVED.
M6S is RECEIVED.
M1E is RECEIVED.
M5S is RECEIVED.
M4T is RECEIVED.
M5H is RECEIVED.
M9B is RECEIVE

In [34]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u,
              'Latitude': latitude, 'Longitude':longitude}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_base.csv')
df_toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
1,M4G,East York,Leaside,43.709060,-79.363452
2,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
3,M1W,Scarborough,Steeles West,43.799525,-79.318389
4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
5,M9P,Etobicoke,Westmount,43.696319,-79.532242
6,M1G,Scarborough,Woburn,43.770992,-79.216917
7,M4J,East York,East Toronto,43.685347,-79.338106
8,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Mimico NE, Old M...",43.636258,-79.498509
9,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
